# **Imports**

In [1]:
import os
os.chdir("../..")
os.chdir(r"src")

import warnings
warnings.filterwarnings("ignore")

from GloVe.glove_functs import *
from GloVe.weights import *

from dask import dataframe as dd
from scipy import sparse
from mittens import Mittens
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

os.chdir("../")

# **Cooccurrence matrix**

In [ ]:
for i in range(14) :

    df = dd.read_csv('data/FinalDataframes/FilteredFinalDataFrame_201'+str(i)+'.csv', dtype={'url': 'object'})
    print('df ouvert'+str(i))
    df = df.compute()
    print('df computed'+str(i))
    df['text'] = df['text'].map(ast.literal_eval)
    df['agenda'] = df['agenda'].map(ast.literal_eval)
    print('df mapped'+str(i))
    
    print('debut après imports'+str(i))
    
    vocab, word2idx = vocab_dic('data/words/Finalwords_201'+str(i)+'.json')
    
    print('après vocab_dic'+str(i))
    
    with open('data/vocabs/vocab_201'+str(i)+'.json', 'w') as f:
        json.dump(vocab, f)
    
    with open('data/word2idx/word2idx_201'+str(i)+'.json', 'w') as f:
        json.dump(word2idx, f)
    
    items = [(j,t) for j,t in enumerate(df['text'])]

    coocc = inter_coocc(items, word2idx)
    
    coocc = coocc.tocsr()
    
    print('dok_array'+str(i))
    
    sparse.save_npz('data/glove_cooccurences/glove_cooccurences_201'+str(i)+'.npz', coocc)

# **GloVe Training**

In [ ]:
original_embedding = glove2dict('data/glove.6B/glove.6B.50d.txt')

for i in range(14) :

    with open('vocab_201'+str(i)+'.json') as f:
        vocab = json.load(f)
        
    cooccurrence = sparse.load_npz('data/glove_cooccurences/glove_cooccurrence_201'+str(i)+'.npz')
    cooccurrence = cooccurrence.toarray()
    
    mittens_model = Mittens(n=50, max_iter=1000)
    
    new_embeddings = mittens_model.fit(
        cooccurrence,
        vocab=vocab,
        initial_embedding_dict= original_embedding)
        
    a = np.array(vocab)
    b =new_embeddings
    c = np.column_stack((a, b))
    np.savetxt('data/embeddings/embeddings_201'+str(i)+'.txt', c, fmt='%s')
    
    ################################################################
    
    original_embedding = glove2dict('data/embeddings/embeddings_201'+str(i)+'.txt')

# **Document embeddings**

In [ ]:
for i in range(14):
    # Path to the GloVe file for a specific year, converted to Word2Vec format
    #glove_file_words = datapath('data/embeddings/embeddings_201'+str(i)+'.txt') #This is the old way of doing it.
    word2vec_glove_file_words = get_tmpfile("format_word2vec.text")
    
    # Convert GloVe format to Word2Vec format
    glove2word2vec('data/embeddings/embeddings_201'+str(i)+'.txt', word2vec_glove_file_words)
    
    # Load the Word2Vec model
    model = KeyedVectors.load_word2vec_format(word2vec_glove_file_words)
    
    # Load the words for which weights need to be computed
    with open('data/words/Finalwords_201'+str(i)+'.json') as f:
        words = json.load(f)
    
    # Compute weights for the words
    weights = get_weights_word2vec(words, a=1e-3)
    
    # Open the corpus for the year and reset its index
    df = standard_opening('data/FinalDataframes/FilteredFinalDataFrame_201'+str(i)+'.csv', True)
    df.reset_index(inplace=True, drop=True)
    
    # Calculate sentence embeddings for each text in the dataframe
    df['sentence_embedding'] = df['text'].apply(get_sentence_embeddings, weights=weights, model=model)
    
    # Filter and organize dataframe columns
    df = df[['text', 'sentence_embedding', 'source', 'party', 'keywords', 'Speaker']]
    df = df[df['sentence_embedding'].notna()]
    
    # Remove the principal component from the sentence embeddings
    b = np.array(df['sentence_embedding'].tolist())
    b = remove_pc(b, npc=1)
    b = b.astype(str)
    df['sentence_embedding'] = b.tolist()
    
    # Save the dataframe with sentence embeddings to a CSV file
    df.to_csv('data/sentence_embeddings/sentence_embeddings_201'+str(i)+'.csv', index=True)
    
    # Also save the text and processed embeddings to a TXT file
    c = np.column_stack((df['text'].apply(phrase), b))
    np.savetxt('data/sentence_embeddings/sentence_embeddings_201'+str(i)+'.txt', c, fmt='%s')

    print(f'end of loop {i}')